#### Spark SQL Queries
- Spark SQL lets you query structured data using standard SQL syntax on top of Apache Spark’s distributed engine. 
- It supports almost everything you know from traditional SQL (SELECT, WHERE, JOIN, GROUP BY, etc.) but at massive scale.

### We can use Spark SQL in Databricks in three ways:
1. Through the DataFrame API and converting it to SQL views
2. Using spark.sql("...") inside a PySpark notebook
3. Using %sql magic commands directly in a notebook cell

In [0]:
# 1. Through the DataFrame API and converting it to SQL views
# Read CSV data
cust_df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/inceptez_catalog/inputdb/customerdata/custs") \
    .toDF("custid","fname","lname","age","profession")
)
cust_df.printSchema()
# Create or replace a temporary SQL view
cust_df.createOrReplaceTempView("v_customer")

# Query using SQL on the temp view
df_result = spark.sql("""
SELECT custid, fname, age, profession
FROM v_customer
WHERE age > 30
ORDER BY age DESC
""")

display(df_result)

In [0]:
# Using spark.sql("...") inside a PySpark notebook
# You can directly query the CSV file using Delta-like SQL syntax
df = spark.sql("""
SELECT * FROM        
(SELECT _c0 as custid,_c1 as fname,_c2 as lname,_c3 as age,_c4 as profession
FROM csv.`dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/custs`)
WHERE profession = 'Lawyer'
AND age > 30
ORDER BY age DESC
""")
df.display()



In [0]:
%sql
-- Create temp view using sql query
CREATE OR REPLACE TEMP VIEW temp_view 
USING CSV
OPTIONS (
  inferSchema "true",
  path "dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/custs"
);

SELECT * FROM        
(SELECT _c0 as custid,_c1 as fname,_c2 as lname,_c3 as age,_c4 as profession
FROM temp_view)
WHERE profession = 'Lawyer'
AND age > 30;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cust_view 
USING CSV
OPTIONS (
  inferSchema "true",
  path "dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/custs"
);

CREATE OR REPLACE TEMP VIEW cust_view1 AS
SELECT * FROM        
(SELECT _c0 as custid,_c1 as fname,_c2 as lname,_c3 as age,_c4 as profession
FROM cust_view);

CREATE OR REPLACE TEMP VIEW txns_view 
USING CSV
OPTIONS (
  inferSchema "true",
  path "dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/txns"
);

CREATE OR REPLACE TEMP VIEW txns_view1 AS
SELECT * FROM        
(SELECT _c0 as txnid,_c1 as txndate,_c2 as custid,_c3 as txnamount,_c4 as prodname,_c5 as category,_c6 as city,_c7 as state,_c8 as paymenttype FROM txns_view);

select * from txns_view1 limit 10;


CREATE OR REPLACE TEMP VIEW final_view AS
SELECT c.fname,c.lname,c.profession, t.txnamount,t.city,t.state FROM cust_view1 c INNER JOIN txns_view1 t ON c.custid = t.custid WHERE c.profession = 'Pilot' AND c.age > 30;

CREATE OR REPLACE TABLE inceptez_catalog.inputdb.tblcustomertxns AS
SELECT * FROM final_view;





In [0]:
%sql
SELECT c.fname,c.lname,c.profession, t.txnamount,t.city,t.state FROM cust_view1 c INNER JOIN txns_view1 t ON c.custid = t.custid WHERE c.profession = 'Pilot' AND c.age > 30;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW temp_view_raw
USING CSV
OPTIONS (
  header "false",
  inferSchema "true",
  path "dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/custs"
);

CREATE OR REPLACE TEMP VIEW temp_view AS
SELECT
  _c0 AS custid,
  _c1 AS fname,
  _c2 AS lname,
  _c3 AS age,
  _c4 AS profession
FROM temp_view_raw;


CREATE OR REPLACE TABLE cust_csv
USING CSV
OPTIONS (
  path '/Volumes/inceptez_catalog/outputdb/customerdata/cust_csv',
  header 'true'
)
AS
SELECT *
FROM temp_view;

-- ============================================================
-- 2. Save temp view data into JSON
-- ============================================================

CREATE OR REPLACE TABLE cust_json
USING JSON
OPTIONS (
  path '/Volumes/inceptez_catalog/outputdb/customerdata/cust_json'
)
AS
SELECT *
FROM temp_view;

-- ============================================================
-- 3. Save temp view data into Parquet
-- ============================================================

CREATE OR REPLACE TABLE cust_parquet
USING PARQUET
OPTIONS (
  path '/Volumes/inceptez_catalog/outputdb/customerdata/cust_parquet'
)
AS
SELECT *
FROM temp_view;